In [59]:
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import AdamW, WarmUp, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import os
import time
import copy
import numpy
import matplotlib.pyplot as plt
import jsonlines

In [60]:
import pandas as pd

In [138]:
data = pd.read_csv('train.csv')

In [124]:
with jsonlines.open('C:/Users/krilo/PycharmProjects/ProphetNet/GENIE/GENIE/datasets/ToTTo/valid_old.jsonl') as f:
    old = [line for line in f.iter()]

In [136]:
pd.DataFrame(old)['src'].iloc[3]

'<page_title> List of rulers of Brittany </page_title> <section_title> House of Montfort </section_title> <table> <cell> Peter II the Simple (Pêr II) 1450–1457 <col_header> Name </col_header> </cell> <cell> Arthur III the Justicier (Arzhur III) 1457–1458 <col_header> Name </col_header> </cell> <cell> 26 December 1458 Nantes aged 65 <col_header> Death </col_header> </cell> </table>'

In [137]:
data['text'].iloc[3]

'<page_title> List of rulers of Brittany </page_title> <section_title> House of Montfort </section_title> <table> <cell> 26 December 1458 Nantes aged 65 <header> Death </header> </cell> <cell> Arthur III the Justicier (Arzhur III) 1457–1458 <header> Name </header> </cell> <cell> Peter II the Simple (Pêr II) 1450–1457 <header> Name </header> </cell> </table>'

In [110]:
old[7699-0] == data['text'].iloc[0]

False

In [117]:
data['text'].iloc[7698]

'<page_title> List of Religious Cultural Monuments of Albania </page_title> <section_title> Durrës County </section_title> <table> <cell> Durrës <header> Location </header> </cell> <cell> Fatih Mosque <header> Name </header> </cell> </table>'

In [118]:
old[1]

'<page_title> Amlogic </page_title> <section_title> Comparison table </section_title> <table> <cell> 1 <header> CPU </header> <header> µarch </header> </cell> <cell> 2011 <header> Sampl. Avail- ability </header> </cell> <cell> 65 nm <header> Fab </header> <header> µarch </header> </cell> <cell> AML8726-M <header> ISA </header> <header> Model Number </header> </cell> <cell> ARMv7-A <header> CPU </header> <header> Cores </header> </cell> <cell> Cortex-A9 <header> CPU </header> <header> Freq. (GHz) </header> </cell> <cell> Mali 400 MP <header> GPU </header> <header> Type </header> </cell> </table>'

In [120]:
pd.DataFrame(old)

,0
0,<page_title> Amlogic </page_title> <section_ti...
1,<page_title> Amlogic </page_title> <section_ti...
2,<page_title> Amlogic </page_title> <section_ti...
3,<page_title> Amlogic </page_title> <section_ti...
4,<page_title> Amlogic </page_title> <section_ti...
...,...
7695,<page_title> Amlogic </page_title> <section_ti...
7696,<page_title> Amlogic </page_title> <section_ti...
7697,<page_title> Amlogic </page_title> <section_ti...
7698,<page_title> Amlogic </page_title> <section_ti...


In [115]:
for i in range(7700):
    print(old[i] == data['text'].iloc[7699-i])
    

True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False

In [63]:
%pwd

'C:\\Users\\krilo\\PycharmProjects\\Lattice\\totto_data'

In [139]:
import jsonlines

items = [{'src' : i[1]['text'], 'trg' : i[1]['summary']} for i in data[['text', 'summary']].iterrows()]

with jsonlines.open('train.jsonl', 'w') as writer:
    writer.write_all(items)

In [23]:
# fout = open('./corrects.txt', 'w', encoding='utf8')
# with jsonlines.open('valid.jsonl') as f:
#     for line in f.iter():
#         fout.write(line['trg']+'\n')
# fout.close()

In [24]:
with open('rank0_gen_seed_101_num1_epoch1.txt', encoding='utf8') as f:
    lines = f.readlines()
    
with open('corrects.txt', encoding='utf8') as f:
    lines = f.readlines()

In [25]:
%%bash
git clone https://github.com/google-research/language.git language_repo
cd language_repo
pip3 install -r language/totto/eval_requirements.txt

Couldn't find program: 'bash'


In [23]:
%cd language_repo
!bash language/totto/totto_eval.sh --prediction_path language/totto/sample/output_sample.txt --target_path language/totto/sample/dev_sample.jsonl

/kaggle/working/language_repo
Running with the following variables:
PREDICTION_PATH   : language/totto/sample/output_sample.txt
TARGET_PATH       : language/totto/sample/dev_sample.jsonl 
BLEURT_CKPT       : unset 
OUTPUT_DIR        : temp
MODE              : test
Creating Output directory.
Cloning moses for BLEU script.
Cloning into 'temp/mosesdecoder'...
remote: Enumerating objects: 148070, done.
remote: Counting objects: 100% (498/498), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 148070 (delta 315), reused 433 (delta 289), pack-reused 147572
Receiving objects: 100% (148070/148070), 129.86 MiB | 19.84 MiB/s, done.
Resolving deltas: 100% (114341/114341), done.
Writing references.
Writing tables in PARENT format.
Preparing predictions.
Writing predictions.
Running detokenizers.
======== EVALUATE OVERALL ========
Computing BLEU (overall)
{
 "name": "BLEU",
 "score": 45.5,
 "signature": "nrefs:3|case:mixed|eff:no|tok:13a|smooth:exp|version:2.0.0",
 "verbose_sco

In [24]:
len(output)

7700

In [25]:
!bash language/totto/totto_eval.sh --prediction_path /kaggle/working/outputs.txt --target_path /kaggle/input/table-to-text-generation-dataset-google-totto/totto_data/totto_dev_data.jsonl --output_dir /kaggle/working/

Running with the following variables:
PREDICTION_PATH   : /kaggle/working/outputs.txt
TARGET_PATH       : /kaggle/input/table-to-text-generation-dataset-google-totto/totto_data/totto_dev_data.jsonl 
BLEURT_CKPT       : unset 
OUTPUT_DIR        : /kaggle/working
MODE              : test
Cloning moses for BLEU script.
Cloning into '/kaggle/working/mosesdecoder'...
remote: Enumerating objects: 148070, done.
remote: Counting objects: 100% (498/498), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 148070 (delta 315), reused 433 (delta 289), pack-reused 147572
Receiving objects: 100% (148070/148070), 129.86 MiB | 22.12 MiB/s, done.
Resolving deltas: 100% (114341/114341), done.
Writing references.
Writing tables in PARENT format.
Preparing predictions.
Writing predictions.
Running detokenizers.
======== EVALUATE OVERALL ========
Computing BLEU (overall)
{
 "name": "BLEU",
 "score": 32.6,
 "signature": "nrefs:3|case:mixed|eff:no|tok:13a|smooth:exp|version:2.0.0",
 "verbo